# optionm

In [ ]:
// // test

open testing

()



## default_value

In [ ]:
inl default_value d = optionm.defaultWith d

()



In [ ]:
// // test

None
|> default_value 3i32
|> _equal 3i32

let rec method0 () : unit =
    let v0 : string = $"_equal / actual: %A{3} / expected: %A{3}"
    ()
method0()



## default_with

In [ ]:
inl default_with fn = function Some x => x | None => fn ()

()



In [ ]:
// // test

None
|> default_with (fun () => 3i32)
|> _equal 3i32

let rec method0 () : unit =
    let v0 : string = $"_equal / actual: %A{3} / expected: %A{3}"
    ()
method0()



## choose

In [ ]:
inl choose forall a b c. (fn : a -> b -> c) (a : option a) (b : option b) : option c =
    match a, b with
    | Some x, Some y => fn x y |> Some
    | _ => None

()



In [ ]:
// // test

(Some 2i32, Some 3)
||> choose (+)
|> _equal (Some 5)

type [<Struct>] US0 =
    | US0_0
    | US0_1 of f1_0 : int32
let rec method0 () : unit =
    let v3 : US0 = US0_1(5)
    let v4 : US0 = US0_1(5)
    let v5 : string = $"_equal / actual: %A{v3} / expected: %A{v4}"
    ()
method0()



## iter

In [ ]:
inl iter fn = function
    | Some x => fn x
    | None => ()

()



In [ ]:
// // test

inl n = mut 1i32
inl fn =
    fun n' =>
        n <- *n + n'
Some 1i32 |> iter fn
None |> iter fn
*n
|> _equal 2i32

type Mut0 = {mutable l0 : int32}
let rec method0 () : unit =
    let v0 : Mut0 = {l0 = 1} : Mut0
    let v1 : int32 = v0.l0
    let v2 : int32 = v1 + 1
    v0.l0 <- v2
    let v3 : int32 = v0.l0
    let v4 : bool = v3 = 2
    let v5 : string = $"_equal / actual: %A{v3} / expected: %A{2}"
    let v6 : bool = v4 = false
    if v6 then
        failwith<unit> v5
method0()

